In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.base import clone

#-------------------------------------------
# Load and preprocess data
#-------------------------------------------
census = pd.read_csv("census2000.csv")
census.rename(columns=lambda c: c[1:] if c.startswith(' ') else c, inplace=True)

# Treatment indicator: T = 1 if male, 0 if female
census['T'] = (census['sex'] == 'M').astype(int)

# Outcome: log wage = log(income/hours)
census['wage'] = census['income'] / census['hours']
census['Y'] = np.log(census['wage'])

# Define features (excluding income, hours, sex, wage, Y)
# Here we use age, marital, race, education as features
# Convert categorical variables to dummies
X = pd.get_dummies(census[['age','marital','race','education']], drop_first=True)
Y = census['Y'].values
T = census['T'].values

# Proportion treated
p_hat = T.mean()

#-------------------------------------------
# Functions for first-step estimation
#-------------------------------------------

def fit_propensity_model(model, X, T):
    """
    Fit propensity score model.
    model: classifier with predict_proba
    """
    model = clone(model)
    model.fit(X, T)
    p_hat_x = model.predict_proba(X)[:,1]
    # truncate probability estimates
    p_hat_x = np.clip(p_hat_x, 1e-3, 1-1e-3)
    return model, p_hat_x

def fit_outcome_model(model, X, Y, T):
    """
    Fit outcome regression model for the control group only.
    model: regressor with predict
    """
    model = clone(model)
    # Fit only on controls
    model.fit(X[T==0], Y[T==0])
    mu0_hat_x = model.predict(X)
    return model, mu0_hat_x

#-------------------------------------------
# Estimators
#-------------------------------------------

def plugin_estimator(T, Y, p_hat, p_hat_x, mu0_hat_x):
    """
    Plug-in estimator of ATT:
    tau_PI = mu1_hat - mu0_hat
    with mu1_hat = (1/n)*sum(T_i * Y_i / p_hat)
    and mu0_hat = (1/n)*sum{ (1-T_i)*p_hat(X_i)*Y_i / [ (1 - p_hat(X_i)) ] }

    p_hat is the average of T (overall proportion treated).
    p_hat_x is the estimated propensity for each unit.
    mu0_hat_x is the estimated outcome regression for the control condition.
    """
    n = len(T)
    mu1_hat = np.sum(T * Y / p_hat) / n
    mu0_hat = np.sum((1 - T) * p_hat_x * Y / (1 - p_hat_x)) / n
    tau_PI = mu1_hat - mu0_hat
    return tau_PI

def doubly_robust_estimator(T, Y, p_hat, p_hat_x, mu0_hat_x):
    """
    Doubly robust estimator of ATT:
    tau_DR = (1/n)*sum( t_i * y_i / p_hat ) 
             - (1/p_hat)* (1/n)* sum( t_i * mu0_hat(x_i) + (1-t_i)*p_hat(x_i)*Y_i/(1 - p_hat(x_i)) )
    """
    n = len(T)
    term1 = np.mean(T * Y / p_hat)
    term2 = (1/(p_hat)) * np.mean( T*mu0_hat_x + (1 - T)*p_hat_x*Y/(1 - p_hat_x) )
    tau_DR = term1 - term2
    return tau_DR

#-------------------------------------------
# Cross-fitting option
#-------------------------------------------

def cross_fit_estimates(X, Y, T, p_hat, prop_model, outcome_model, n_splits=2):
    """
    Perform cross-fitting: split data into folds, 
    estimate nuisance functions on one fold and evaluate on the other.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    p_hat_x = np.zeros(len(T))
    mu0_hat_x = np.zeros(len(T))

    for train_idx, test_idx in kf.split(X):
        # Fit models on training fold
        prop_model_fold, _ = fit_propensity_model(prop_model, X[train_idx], T[train_idx])
        outcome_model_fold, _ = fit_outcome_model(outcome_model, X[train_idx], Y[train_idx], T[train_idx])

        # Predict on test fold
        p_hat_x[test_idx] = prop_model_fold.predict_proba(X[test_idx])[:,1]
        mu0_hat_x[test_idx] = outcome_model_fold.predict(X[test_idx])

    # Compute estimators
    tau_PI = plugin_estimator(T, Y, p_hat, p_hat_x, mu0_hat_x)
    tau_DR = doubly_robust_estimator(T, Y, p_hat, p_hat_x, mu0_hat_x)
    return tau_PI, tau_DR

def single_fit_estimates(X, Y, T, p_hat, prop_model, outcome_model):
    """
    No cross-fitting: fit on entire sample.
    """
    _, p_hat_x = fit_propensity_model(prop_model, X, T)
    _, mu0_hat_x = fit_outcome_model(outcome_model, X, Y, T)

    tau_PI = plugin_estimator(T, Y, p_hat, p_hat_x, mu0_hat_x)
    tau_DR = doubly_robust_estimator(T, Y, p_hat, p_hat_x, mu0_hat_x)
    return tau_PI, tau_DR

#-------------------------------------------
# Model configurations
#-------------------------------------------
# We consider various methods for p_hat(x) and mu0_hat(x):
# For demonstration: logistic regression, random forest, neural net, lasso, parametric linear.

propensity_models = {
    "LogisticRegression": LogisticRegression(solver='lbfgs', max_iter=1000),
    "RandomForestClassifier": RandomForestClassifier(n_estimators=100, random_state=42),
    "NeuralNetworkClassifier": MLPClassifier(hidden_layer_sizes=(32,16), max_iter=500, random_state=42)
}

outcome_models = {
    "LinearRegression": LinearRegression(),
    "RandomForestRegressor": RandomForestRegressor(n_estimators=100, random_state=42),
    "NeuralNetworkRegressor": MLPRegressor(hidden_layer_sizes=(32,16), max_iter=500, random_state=42),
    "LassoRegression": Lasso(alpha=0.1)
}

# Convert X to numpy for convenience
X_np = X.values

#-------------------------------------------
# Run the analyses and show results
#-------------------------------------------
results = []

for p_name, p_model in propensity_models.items():
    for m_name, m_model in outcome_models.items():
        # Without cross-fitting
        tau_PI_no_cf, tau_DR_no_cf = single_fit_estimates(X_np, Y, T, p_hat, p_model, m_model)

        # With cross-fitting
        tau_PI_cf, tau_DR_cf = cross_fit_estimates(X_np, Y, T, p_hat, p_model, m_model, n_splits=2)

        results.append({
            "Propensity_Model": p_name,
            "Outcome_Model": m_name,
            "CrossFitting": False,
            "Tau_PI": tau_PI_no_cf,
            "Tau_DR": tau_DR_no_cf
        })

        results.append({
            "Propensity_Model": p_name,
            "Outcome_Model": m_name,
            "CrossFitting": True,
            "Tau_PI": tau_PI_cf,
            "Tau_DR": tau_DR_cf
        })

# Convert results to DataFrame and print
results_df = pd.DataFrame(results)
print(results_df)

# This results_df now contains the ATT estimates for both the PI and DR estimators,
# with and without cross-fitting, and for different first-step models.
# You can further analyze, plot, or discuss these results as needed.

/var/folders/03/lg9h_1xd7nd8d90d67rgwtf40000gn/T/ipykernel_9745/1006876555.py:77: RuntimeWarning: divide by zero encountered in divide
  mu0_hat = np.sum((1 - T) * p_hat_x * Y / (1 - p_hat_x)) / n
/var/folders/03/lg9h_1xd7nd8d90d67rgwtf40000gn/T/ipykernel_9745/1006876555.py:77: RuntimeWarning: invalid value encountered in divide
  mu0_hat = np.sum((1 - T) * p_hat_x * Y / (1 - p_hat_x)) / n
/opt/homebrew/anaconda3/envs/finm/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/var/folders/03/lg9h_1xd7nd8d90d67rgwtf40000gn/T/ipykernel_9745/1006876555.py:89: RuntimeWarning: divide by zero encountered in divide
  term2 = (1/(p_hat)) * np.mean( T*mu0_hat_x + (1 - T)*p_hat_x*Y/(1 - p_hat_x) )
/var/folders/03/lg9h_1xd7nd8d90d67rgwtf40000gn/T/ipykernel_9745/1006876555.py:89: RuntimeWarning: invalid value encountered in divide
  term2 = (1/(p_hat)) * np.mean( T*mu0_hat_x + (1 - 

           Propensity_Model           Outcome_Model  CrossFitting    Tau_PI  \
0        LogisticRegression        LinearRegression         False  0.874867   
1        LogisticRegression        LinearRegression          True  0.871097   
2        LogisticRegression   RandomForestRegressor         False  0.874867   
3        LogisticRegression   RandomForestRegressor          True  0.871097   
4        LogisticRegression  NeuralNetworkRegressor         False  0.874867   
5        LogisticRegression  NeuralNetworkRegressor          True  0.871097   
6        LogisticRegression         LassoRegression         False  0.874867   
7        LogisticRegression         LassoRegression          True  0.871097   
8    RandomForestClassifier        LinearRegression         False  0.958338   
9    RandomForestClassifier        LinearRegression          True       NaN   
10   RandomForestClassifier   RandomForestRegressor         False  0.958338   
11   RandomForestClassifier   RandomForestRegressor 

In [16]:
results_df.to_csv("q2c-data/preliminary_2_results.csv", index=False)

In [17]:
results_df

,Propensity_Model,Outcome_Model,CrossFitting,Tau_PI,Tau_DR
0,LogisticRegression,LinearRegression,False,0.874867,-1.995883
1,LogisticRegression,LinearRegression,True,0.871097,-2.004188
2,LogisticRegression,RandomForestRegressor,False,0.874867,-1.951904
3,LogisticRegression,RandomForestRegressor,True,0.871097,-1.958615
4,LogisticRegression,NeuralNetworkRegressor,False,0.874867,-2.220714
5,LogisticRegression,NeuralNetworkRegressor,True,0.871097,-2.090103
6,LogisticRegression,LassoRegression,False,0.874867,-2.036195
7,LogisticRegression,LassoRegression,True,0.871097,-2.042983
8,RandomForestClassifier,LinearRegression,False,0.958338,-1.843984
9,RandomForestClassifier,LinearRegression,True,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.base import clone

#-------------------------------------------
# Load and preprocess data
#-------------------------------------------
census = pd.read_csv("census2000.csv")
census.rename(columns=lambda c: c[1:] if c.startswith(' ') else c, inplace=True)

# Treatment indicator: T = 1 if male, 0 if female
census['T'] = (census['sex'] == 'M').astype(int)

# Outcome: log wage = log(income/hours)
census['wage'] = census['income'] / census['hours']
census['Y'] = np.log(census['wage'])

# Define features (excluding income, hours, sex, wage, Y)
# Here we use age, marital, race, education as features
# Convert categorical variables to dummies
X = pd.get_dummies(census[['age','marital','race','education']], drop_first=True)
Y = census['Y'].values
T = census['T'].values

# Proportion treated
p_hat_overall = T.mean()

#-------------------------------------------
# Functions for first-step estimation
#-------------------------------------------

def fit_propensity_model(model, X, T):
    """
    Fit propensity score model.
    model: classifier with predict_proba
    """
    model = clone(model)
    model.fit(X, T)
    p_hat_x = model.predict_proba(X)[:,1]
    # Truncate probability estimates
    p_hat_x = np.clip(p_hat_x, 1e-3, 1-1e-3)
    return model, p_hat_x

def fit_outcome_model(model, X, Y, T):
    """
    Fit outcome regression model for the control group only.
    model: regressor with predict
    """
    model = clone(model)
    # Check if there are control units
    if np.sum(T == 0) == 0:
        raise ValueError("No control units in the training fold.")
    model.fit(X[T==0], Y[T==0])
    mu0_hat_x = model.predict(X)
    
    # Check for NaNs or infinite values
    if np.any(np.isnan(mu0_hat_x)) or np.any(np.isinf(mu0_hat_x)):
        raise ValueError("mu0_hat_x contains NaNs or infinite values.")
    
    return model, mu0_hat_x

#-------------------------------------------
# Estimators
#-------------------------------------------

def plugin_estimator(T, Y, p_hat_x, mu0_hat_x):
    """
    Plug-in estimator of ATT:
    tau_PI = mu1_hat - mu0_hat
    with mu1_hat = (1/n)*sum(T_i * Y_i / p_hat_x_i)
    and mu0_hat = (1/n)*sum{ (1-T_i)*p_hat_x_i*Y_i / [ (1 - p_hat_x_i) ] }
    """
    n = len(T)
    mu1_hat = np.sum(T * Y / p_hat_x) / n
    mu0_hat = np.sum((1 - T) * p_hat_x * Y / (1 - p_hat_x)) / n
    tau_PI = mu1_hat - mu0_hat
    return tau_PI

def doubly_robust_estimator(T, Y, p_hat_x, mu0_hat_x):
    """
    Doubly robust estimator of ATT:
    tau_DR = (1/n)*sum( T_i * Y_i / p_hat_x_i ) 
             - (1/n)*sum( T_i * mu0_hat_x_i / p_hat_x_i + (1 - T_i) * p_hat_x_i * Y_i / (1 - p_hat_x_i) )
    """
    term1 = np.mean(T * Y / p_hat_x)
    term2 = np.mean(T * mu0_hat_x / p_hat_x + (1 - T) * p_hat_x * Y / (1 - p_hat_x))
    tau_DR = term1 - term2
    return tau_DR

#-------------------------------------------
# Cross-fitting option
#-------------------------------------------

def cross_fit_estimates(X, Y, T, prop_model, outcome_model, n_splits=2):
    """
    Perform cross-fitting: split data into stratified folds, 
    estimate nuisance functions on one fold and evaluate on the other.
    """
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    p_hat_x = np.zeros(len(T))
    mu0_hat_x = np.zeros(len(T))

    for train_idx, test_idx in kf.split(X, T):
        # Fit models on training fold
        prop_model_fold, p_hat_x_fold = fit_propensity_model(prop_model, X[train_idx], T[train_idx])
        outcome_model_fold, mu0_hat_x_fold = fit_outcome_model(outcome_model, X[train_idx], Y[train_idx], T[train_idx])

        # Predict on test fold
        p_hat_x[test_idx] = prop_model_fold.predict_proba(X[test_idx])[:,1]
        mu0_hat_x[test_idx] = outcome_model_fold.predict(X[test_idx])

    # Ensure p_hat_x is clipped properly
    p_hat_x = np.clip(p_hat_x, 1e-3, 1-1e-3)
    
    # Verify no propensity scores are at the bounds
    assert np.all(p_hat_x >= 1e-3) and np.all(p_hat_x <= 1-1e-3), "Propensity scores out of bounds after clipping."

    # Compute estimators using unit-specific propensity scores
    tau_PI = plugin_estimator(T, Y, p_hat_x, mu0_hat_x)
    tau_DR = doubly_robust_estimator(T, Y, p_hat_x, mu0_hat_x)
    return tau_PI, tau_DR

def single_fit_estimates(X, Y, T, prop_model, outcome_model):
    """
    No cross-fitting: fit on entire sample.
    """
    _, p_hat_x = fit_propensity_model(prop_model, X, T)
    _, mu0_hat_x = fit_outcome_model(outcome_model, X, Y, T)

    # Ensure p_hat_x is clipped properly
    p_hat_x = np.clip(p_hat_x, 1e-3, 1-1e-3)
    assert np.all(p_hat_x >= 1e-3) and np.all(p_hat_x <= 1-1e-3), "Propensity scores out of bounds after clipping."
    assert not np.any(np.isnan(mu0_hat_x)), "mu0_hat_x contains NaNs."
    assert not np.any(np.isinf(mu0_hat_x)), "mu0_hat_x contains infinite values."

    tau_PI = plugin_estimator(T, Y, p_hat_x, mu0_hat_x)
    tau_DR = doubly_robust_estimator(T, Y, p_hat_x, mu0_hat_x)
    return tau_PI, tau_DR

#-------------------------------------------
# Model configurations
#-------------------------------------------
# We consider various methods for p_hat(x) and mu0_hat(x):
# For demonstration: logistic regression, random forest, neural net, lasso, parametric linear.

propensity_models = {
    "LogisticRegression": LogisticRegression(solver='lbfgs', max_iter=1000),
    "RandomForestClassifier": RandomForestClassifier(n_estimators=100, random_state=42),
    "NeuralNetworkClassifier": MLPClassifier(hidden_layer_sizes=(32,16), max_iter=1000, random_state=42)
}

outcome_models = {
    "LinearRegression": LinearRegression(),
    "RandomForestRegressor": RandomForestRegressor(n_estimators=100, random_state=42),
    "NeuralNetworkRegressor": MLPRegressor(hidden_layer_sizes=(32,16), max_iter=1000, random_state=42),
    "LassoRegression": Lasso(alpha=0.1)
}

# Convert X to numpy for convenience
X_np = X.values

#-------------------------------------------
# Run the analyses and show results
#-------------------------------------------
results = []

for p_name, p_model in propensity_models.items():
    for m_name, m_model in outcome_models.items():
        # Without cross-fitting
        try:
            tau_PI_no_cf, tau_DR_no_cf = single_fit_estimates(X_np, Y, T, p_model, m_model)
        except ValueError as e:
            tau_PI_no_cf, tau_DR_no_cf = np.nan, np.nan
            print(f"Error in single_fit_estimates with Propensity Model: {p_name}, Outcome Model: {m_name} - {e}")

        # With cross-fitting
        try:
            tau_PI_cf, tau_DR_cf = cross_fit_estimates(X_np, Y, T, p_model, m_model, n_splits=2)
        except ValueError as e:
            tau_PI_cf, tau_DR_cf = np.nan, np.nan
            print(f"Error in cross_fit_estimates with Propensity Model: {p_name}, Outcome Model: {m_name} - {e}")

        results.append({
            "Propensity_Model": p_name,
            "Outcome_Model": m_name,
            "CrossFitting": False,
            "Tau_PI": tau_PI_no_cf,
            "Tau_DR": tau_DR_no_cf
        })

        results.append({
            "Propensity_Model": p_name,
            "Outcome_Model": m_name,
            "CrossFitting": True,
            "Tau_PI": tau_PI_cf,
            "Tau_DR": tau_DR_cf
        })

# Convert results to DataFrame and display
results_df = pd.DataFrame(results)
print(results_df)

           Propensity_Model           Outcome_Model  CrossFitting    Tau_PI  \
0        LogisticRegression        LinearRegression         False  0.896997   
1        LogisticRegression        LinearRegression          True  0.895288   
2        LogisticRegression   RandomForestRegressor         False  0.896997   
3        LogisticRegression   RandomForestRegressor          True  0.895288   
4        LogisticRegression  NeuralNetworkRegressor         False  0.896997   
5        LogisticRegression  NeuralNetworkRegressor          True  0.895288   
6        LogisticRegression         LassoRegression         False  0.896997   
7        LogisticRegression         LassoRegression          True  0.895288   
8    RandomForestClassifier        LinearRegression         False  0.912058   
9    RandomForestClassifier        LinearRegression          True -3.515407   
10   RandomForestClassifier   RandomForestRegressor         False  0.912058   
11   RandomForestClassifier   RandomForestRegressor 

In [5]:
census.head(10)

,age,sex,marital,race,education,income,hours,T,wage,Y
0,48,M,Married,White,3.hsgrad,52000,2600,1,20.000000,2.995732
1,24,M,Divorced,White,2.high,35000,2080,1,16.826923,2.822980
2,19,F,Single,Black,3.hsgrad,2400,240,0,10.000000,2.302585
3,18,M,Single,Black,2.high,6100,1500,1,4.066667,1.402824
4,28,M,Married,Other,4.assoc,22000,2080,1,10.576923,2.358675
5,18,F,Single,Black,3.hsgrad,2400,700,0,3.428571,1.232144
6,40,F,Single,Black,5.bachs,23100,2080,0,11.105769,2.407465
7,47,M,Divorced,White,4.assoc,23000,2080,1,11.057692,2.403126
8,37,M,Married,White,4.assoc,32000,2080,1,15.384615,2.733368
9,43,F,Married,White,4.assoc,20000,2080,0,9.615385,2.263364
